# `texts` as Texts

In [1]:
# IMPORTS
import json
import nltk
import re, spacy, textacy
import numpy as np
import pandas as pd

import random
# import matplotlib.pyplot as plt

In [34]:
import plotly.graph_objects as go

In [2]:
# DATA
with open("texts.json", 'r') as f:
    texts = json.load(f)

# Check to see that the data loaded as expected
texts[30:35]

[' Anyway... maybe I shouldn’t be so serious with you. Maybe I should just ask you about your day and make you smile. I do tend to get a bit deep, though — that is my curse lol. I’m very playful as well, however 🙈\n ',
 ' Idk-  I don’t really try to meet people.  I just work, come home, sleep &amp;amp; then do it again the next day.  Maybe that’s some of the problem.  Who knows... I’m starting to just not gaf anymore.  Maybe a race war will kick off &amp;amp; I’ll have purpose then.  Fug it\n ',
 ' I am nice.  I’m a good person.  I mean, I’m annoying at times I’m sure &amp;amp; I can get bitchy like any woman does-  but I’m not a cunt &amp;amp; I’m pretty chill.  I’ve never been with a non white.  I’ve only been with 2 people in my life... both were white.  I don’t have herpes lmao!    I think I’m pretty average as far as looks go.  I’m not some Swedish super model, but I’m not fugly.  I’m in shape.  I don’t have a nigger ass.   Idk-  I’m just a normal average lass kicking off my 30’s 

In [3]:
# This uses standard libraries (no imports needed)
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
# Remove the HTML
plaintexts = [strip_tags(i) for i in texts]

In [5]:
# Change ampersands to "and"s
plaintexts = [x.replace("&amp;", "and") for x in plaintexts]
plaintexts = [y.replace("\n", ".") for y in plaintexts]

In [6]:
plaintexts[1401]

' get buried undet the ice. '

In [7]:
# Tokenize all the texts
words = [ nltk.tokenize.word_tokenize(i.lower()) for i in plaintexts ]

# And check to see that went according to plan
print(words[20:22])

[['are', 'you', 'working', 'today', ',', 'beautiful', 'girl', ',', 'or', 'are', 'you', 'hanging', 'with', 'fam', '?', '.'], ['i', '’', 'm', 'working', '.', 'the', 'only', 'days', 'i', 'have', 'off', 'for', 'the', 'next', '2', 'weeks', 'is', 'christmas', 'day', 'and', 'new', 'year', '’', 's', 'day', '.', ':', '(', '.', 'i', 'need', 'to', 'get', 'another', 'job', '.', 'this', 'one', 'is', 'killing', 'me', '.']]


With all our texts now rendered as a lists of tokens, let's take a look at how long the texts are. We're first just going to get a list of the token counts, then we will determine the texts that fall within number ranges that are generated by some initial hand inspection. My logic is, for example, that texts that are less than 20 words are not going to offer much in the way of narrative, and, as it turns out, the longer texts are stuffed with HTML.

In [8]:
word_0 =  [ i for i in words if len(i) <  20 ]
word_20 = [ i for i in words if 20 < len(i) <  50 ]
word_50 = [ i for i in words if 50 < len(i) < 100 ]
word100 = [ i for i in words if 100 < len(i) < 200 ]
word200 = [ i for i in words if 200 < len(i) < 500 ]
word500 = [ i for i in words if len(i) > 500 ]

In [9]:
# Adding up individuals
all_sets = len(word_0) + len(word_20) + len(word_50) + len(word100) + len(word200) + len(word500)
print(f"Sum of all sets: {all_sets}")
print(f"This is {70596 - all_sets} fewer than given.")

workable_sets = len(word_20) + len(word_50) + len(word100) + len(word200)
print(f"Sum of individual sets: {workable_sets}")

# Making one composite
workable = [ i for i in words if 20 < len(i) < 500 ]
print(f"Composite set length: {len(workable)}")

# Doing the math
print(f"Difference between all sets and composite set: \
{workable_sets - len(workable)}")

Sum of all sets: 69362
This is 1234 fewer than given.
Sum of individual sets: 18549
Composite set length: 18784
Difference between all sets and composite set: -235


*Sigh*. 1179-1234 texts are missing! For now, I am not going to worry about it. I have enough texts with which to work.

In [10]:
# Let's sample 3 texts from the longer list:
for i in random.sample(workable, 3):
    print(" ".join(i))

is that your sexy bottle and hand ? rethinking your 2 hr off me . and your late nights . and me not sleeping . it 's probably between your legs that you constantly are squeezing talking dirty to your other guys on gab . at least i 'm getting squeezed in the closest spot . red raspberries home . cream , licking liquors .
i live outside of a little town in east texas . i know these patriots here are hitting at the bit to take care of business if it comes down to it . each county has an organized militia . i am in a great area with well water , woods , garden and plenty of assistance . these people already network to help one another . does n't mean they are not temperamental and a bit rough around the edges but good people ..
do you know how to buy bitcoin ? i ’ d like to support gab but you can only pay by us check or bitcoin . visa and mastercard shut down all attempts by gab to use merchant services ..


In [11]:
import pickle

with open("workable.pkl", "wb") as fp:
    pickle.dump(workable, fp)

# ALTERNATE FORMAT (but file sizes twice as big)
# import json
# with open("workable.json", 'w') as f:
#     json.dump(workable, f, indent=2) 

## Part 2: Actants and Actions

In [15]:
# Load the Space pipeline to be used
nlp = spacy.load('en_core_web_sm')

# Use the pipe method to feed documents 
docs = list(nlp.pipe(texts))

In [16]:
def createSVOs(doc, svo_list):
    # Create the list of tuples for the document
    svotriples = list(textacy.extract.triples.subject_verb_object_triples(doc))
    # Convert to list of dictionaries
    for item in svotriples:
        svo_list.append(
            {
                'subject': str(item[0][-1]), 
                'verb': str(item[1][-1]), 
                'object': str(item[2])
            }
        )

In [18]:
# Create the list
svos = []

# Populate the list
for doc in docs:
    createSVOs(doc, svos) 

In [26]:
# Convert the lists to dataframes
df = pd.DataFrame(svos)

df.shape
df.head()

,subject,verb,object
0,i,want,"[to, ask, you, somthing, please, \n ]"
1,i,see,[it]
2,i,see,[it]
3,i,opened,[page]
4,I,see,[it]


In [21]:
from nltk.stem import WordNetLemmatizer

In [25]:
# Instantiate the lemmatizer
wnl = WordNetLemmatizer()

df.verb = df.verb.map(lambda word: wnl.lemmatize(word, pos="v"))

In [27]:
# Save to CSV files 
# >>> Commented out once run
# df.to_csv("svos.csv")

In [36]:
def sankay (df):
    # Merge the two columns of subjects and verbs
    nodes = np.unique(df[["subject", "verb"]], axis=None)
    # Assign each node a unique ID
    nodes = pd.Series(index=nodes, data=range(len(nodes)))
    # Use Ploty's Sankey trace (which is really hard to understand)
    fig = go.Figure(
        go.Sankey(
            node={"label": nodes.index},
            link={
                "source": nodes.loc[df["subject"]],
                "target": nodes.loc[df["verb"]],
                # "value": df["obs"],
            },
        )
    )
    fig.show()

In [37]:
nodes = np.unique(df[["subject", "verb"]], axis=None)

nodes = pd.Series(index=nodes, data=range(len(nodes)))

# Merge the two columns of subjects and verbs
nodes = np.unique(df[["subject", "verb"]], axis=None)
# Assign each node a unique ID
nodes = pd.Series(index=nodes, data=range(len(nodes)))

In [39]:
sankay(df)